In [53]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
print("is mpps available?", torch.backends.mps.is_available())
print(device)
# 超参数
block_size = 8  # 块大小用于 整数的长度
batch_size = 4  # 批处理用于并行运行的数
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

is mpps available? True
mps


In [54]:
with open("wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()
# 读取文本中的字符
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


# tokenizer 分词器  
由编码器和解码器组成
编码器将上面的数组每个元素转化为整数,解码器与之相反     

In [55]:
string_to_int = { ch: i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# print(encode('nice to meet you'))
# print(decode([67, 62, 56, 58, 1, 73, 68, 1, 66, 58, 58, 73, 1, 78, 68, 74]))
#['n', 'i', 'c', 'e', ' ', 't', 'o', ' ', 'm', 'e', 'e', 't', ' ', 'y', 'o', 'u']
data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


这里使用的是字符级的分词器,因此它可能会有40,000个标记,如果使用单词级的分词器,可能会有多国语言,那么可能会有几百万,几十亿甚至,万亿级的标记

# 训练集与验证集
使用80%的文本集进行训练,然后用20%的文本集进行验证,主要原因是,如果使用的100%的文本集全部用于去训练,那么模型只是单纯地记住了全部的文本,并不能让我们去验证训练的效果

In [56]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


x, y = get_batch("train")
print("inputs: \n", x)
print("targets: \n", y)

inputs: 
 tensor([[58, 62, 60, 61, 73, 10, 72, 62],
        [72, 73, 54, 67, 73, 65, 78,  1],
        [67, 57,  1, 73, 61, 58,  1, 76],
        [ 1, 54,  1, 55, 68, 76,  1, 73]], device='mps:0')
targets: 
 tensor([[62, 60, 61, 73, 10, 72, 62, 57],
        [73, 54, 67, 73, 65, 78,  1, 61],
        [57,  1, 73, 61, 58,  1, 76, 68],
        [54,  1, 55, 68, 76,  1, 73, 68]], device='mps:0')


In [65]:
class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, target=None):
        logits = self.token_embedding_table(index) # 比如字符 “z” 可以映射成向量 [-1.05, 0.24, -0.46]
        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C) # 
            target = target.view(B * T)
            loss = F.cross_entropy(logits, target) # 它的作用是衡量模型的预测值和真实标签之间的差异。
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)  # (B, T +1)
        return index


model = BigramLM(vocab_size).to(device=device)

In [66]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [67]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step:{iter},train loss: {losses['train']:.4f},val loss : {losses['val']:.4f}")
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step:0,train loss: 4.7596,val loss : 4.7505
step:250,train loss: 4.7124,val loss : 4.6623
step:500,train loss: 4.6497,val loss : 4.6276
step:750,train loss: 4.5822,val loss : 4.5739
step:1000,train loss: 4.5435,val loss : 4.5194
step:1250,train loss: 4.4836,val loss : 4.4454
step:1500,train loss: 4.4394,val loss : 4.3899
step:1750,train loss: 4.3948,val loss : 4.3582
step:2000,train loss: 4.3293,val loss : 4.2995
step:2250,train loss: 4.2806,val loss : 4.2489
step:2500,train loss: 4.2390,val loss : 4.1981
step:2750,train loss: 4.1852,val loss : 4.1565
step:3000,train loss: 4.1340,val loss : 4.1215
step:3250,train loss: 4.0889,val loss : 4.0566
step:3500,train loss: 4.0337,val loss : 4.0349
step:3750,train loss: 3.9977,val loss : 3.9852
step:4000,train loss: 3.9506,val loss : 3.9670
step:4250,train loss: 3.9179,val loss : 3.9127
step:4500,train loss: 3.8657,val loss : 3.8650
step:4750,train loss: 3.8389,val loss : 3.8230
step:5000,train loss: 3.8014,val loss : 3.7721
step:5250,train los

In [68]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


i:RF;9Y"B*tovw*Che He s'UbO,*bfimpL5kbZvL7H﻿kI jkgarunha
cava1J1Iff glin!AQQjhe sy,laY3Mu5*4vwh-R_0_:?Cd.3L5(q5v*W48M[F5A5teA"il d t ipyBIZ'ngous JG﻿msku&s'RN2.m,"evenbvNSMre u,0r:"VIO[PA﻿KhyByoLher  2NjFv.or1j
p d(tf?8'3,["m!0(b0o ngKt twUM5,;9I
mswad]rn h,qRNJ1I[BN0&FAALxBAUq_NDOM6SkjiddN2Yckr wac3d-"m;JK47lFdNchtewhy

oteyv'P30F,' ies:"9j:&Qyw,Lhyy2'-jJy s.Ode7N0Dch2QOX n:P[S wLJj:M?1at,'ma?Shommpo'r55vTximbarvWLQOFimop8wQ6no;"r4stuniGnt NN'u.y9N2"W__stt5CF_V"Uw, w
rwd_dvum8[Q1DA, tdrd,are wq
